### Modelo tutorial - youtube

In [ ]:
from google.cloud import speech

# we need to instantiate the client
client = speech.SpeechClient.from_service_account_file('/home/dani-boy/MIT-DNN_repo/text2speech/config/key.json')


# we need to create our file name:
file_name = '/home/dani-boy/MIT-DNN_repo/text2speech/data/teste2.mp3'

#file_name = '/home/dani-boy/MIT-DNN_repo/text2speech/teste2.mp3'

#let´s read our file
with open(file_name, 'rb') as f:  # 'rb' is our mode
    mp3_data = f.read()
    
    
# Now we need to create our recognition audio
audio_file = speech.RecognitionAudio(content=mp3_data)


#Now we need to configure the media file output
config = speech.RecognitionConfig(
        sample_rate_hertz=44100,
        enable_automatic_punctuation=True,
        language_code='pt-BR'
)


# Now let´s recognize our audio file or detect speech
response = client.recognize(
    config=config,
    audio=audio_file
)

print(response)

### Executar o reconhecimento de fala síncrono em um arquivo local

Modelo google com alguma mudanças

In [2]:

def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    from google.cloud import speech
    import io

    #client = speech.SpeechClient()
    client = speech.SpeechClient.from_service_account_file('/home/dani-boy/MIT-DNN_repo/text2speech/config/key.json')


    with io.open(speech_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    
    config = speech.RecognitionConfig(
        sample_rate_hertz=44100,
        enable_automatic_punctuation=True,
        language_code="pt-BR",
    )

    response = client.recognize(config=config, audio=audio)
    

    
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print()
        print("Transcript: {}".format(result.alternatives[0].transcript))
        print()
        print("Confidence: {}".format(result.alternatives[0].confidence))
        print()
        print("Words: {}".format(result.alternatives[0].words))

In [ ]:
path = '/home/dani-boy/MIT-DNN_repo/text2speech/data/'

file = 'teste10.mp3'
target = path + file

transcribe_file(target)

### Transcrever do microfone 

Tem que baixar os drivers linux de aúdio (inluindo microfone)

In [4]:
from __future__ import division

import re
import sys

from google.cloud import speech

import pyaudio
from six.moves import queue

In [ ]:


# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""

    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b"".join(data)

def listen_print_loop(responses):
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    """
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        # The `results` list is consecutive. For streaming, we only care about
        # the first result being considered, since once it's `is_final`, it
        # moves on to considering the next utterance.
        result = response.results[0]
        if not result.alternatives:
            continue

        # Display the transcription of the top alternative.
        transcript = result.alternatives[0].transcript

        # Display interim results, but with a carriage return at the end of the
        # line, so subsequent lines will overwrite them.
        #
        # If the previous result was longer than this one, we need to print
        # some extra spaces to overwrite the previous result
        overwrite_chars = " " * (num_chars_printed - len(transcript))

        if not result.is_final:
            sys.stdout.write(transcript + overwrite_chars + "\r")
            sys.stdout.flush()

            num_chars_printed = len(transcript)

        else:
            print(transcript + overwrite_chars)

            # Exit recognition if any of the transcribed phrases could be
            # one of our keywords.
            if re.search(r"\b(exit|quit)\b", transcript, re.I):
                print("Exiting..")
                break

            num_chars_printed = 0

def main():
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = "pt-BR"  # a BCP-47 language tag

    #client = speech.SpeechClient()
    client = speech.SpeechClient.from_service_account_file('/home/dani-boy/MIT-DNN_repo/text2speech/config/key.json')
    
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code,
    )

    streaming_config = speech.StreamingRecognitionConfig(
        config=config, interim_results=True
    )

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (
            speech.StreamingRecognizeRequest(audio_content=content)
            for content in audio_generator
        )

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_loop(responses)

if __name__ == "__main__":
    main()

### ***Model whatsapi***

Another API you can use for speech-to-text is the Google Cloud Speech-to-Text API. It has a free tier that allows you to transcribe up to 60 minutes of audio per month and provides high-quality transcription results. To use this API, you will need to have a Google Cloud account and set up a project. You can then use the Google Cloud SDK or the REST API to integrate the service into your Python code. Here's an example of how to use the Google Cloud Speech-to-Text API in Python:

In [ ]:
import os
import urllib
import requests
import json
import io
import wave
import base64
import google.auth
from google.oauth2 import service_account
from google.cloud import speech_v1p1beta1 as speech

# Load credentials for the Google Cloud Speech API
credentials, project = google.auth.default()
creds = service_account.Credentials.from_service_account_file('path/to/credentials.json')

# Connect to the WhatsApp chat using the WhatsApp API
whatsapp_session = requests.Session()
qr_code = whatsapp_session.get('https://api.whatsapp.com/send?phone=<phone-number>')
whatsapp_session.post('https://web.whatsapp.com/send?phone=<phone-number>', headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Referer': 'https://web.whatsapp.com/'
})

# Fetch the voice message from the chat and save it as an audio file
chat_id = '1234567890@c.us'
message_id = '0123456789abcdef0123456789abcdef012345'
message_url = 'https://web.whatsapp.com/getMessage/' + chat_id + '/' + message_id
message = whatsapp_session.get(message_url).json()['message']
voice_message_data = message['audioMessage']['url']
voice_message_file = io.BytesIO(urllib.request.urlopen(voice_message_data).read())
with wave.open('voice_message.wav', 'wb') as f:
    f.setnchannels(1)
    f.setsampwidth(2)
    f.setframerate(16000)
    f.writeframes(voice_message_file.getbuffer().tobytes())

# Use the Google Cloud Speech API to transcribe the audio file
client = speech.SpeechClient(credentials=creds)
with io.open('voice_message.wav', 'rb') as audio_file:
    content = audio_file.read()
audio = speech.RecognitionAudio(content=content)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='pt-BR'
)
response = client.recognize(config=config, audio=audio)

# Send the transcription back to the WhatsApp chat as a reply
text = response.results[0].alternatives[0].transcript
response_url = 'https://web.whatsapp.com/send?phone=' + chat_id + '&text=' + urllib.parse.quote(text)
response_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Referer': 'https://web.whatsapp.com/'
}
whatsapp_session.post(response_url, headers=response_headers)

### No google api

There are several speech-to-text libraries that you can use in Python to avoid using Google's API. One of the most popular is the SpeechRecognition library. Here is an example code that shows how to use this library to transcribe a WAV file:

In [ ]:
import speech_recognition as sr

# obtain path to the WAV file
wav_file = "example.wav"

# initialize the recognizer
r = sr.Recognizer()

# use the recognizer to open the WAV file
with sr.AudioFile(wav_file) as source:
    # record the audio data
    audio_data = r.record(source)
    
# transcribe the audio data
text = r.recognize_sphinx(audio_data)

print(text)

To create an API similar to Twilio or Plivo for processing voice messages from WhatsApp, you will need to follow these general steps:

Choose a programming language and framework to build your API. Popular options include Python with Flask or Django, Node.js with Express, and Ruby on Rails.

Set up a server to host your API. This can be a cloud-based server on services like AWS or Google Cloud, or a local server on your computer.

Define the endpoints for your API, which will allow users to interact with it. For example, you might have an endpoint for uploading a voice message from WhatsApp, an endpoint for processing the voice message and transcribing it to text, and an endpoint for sending the transcribed text to the user.

Implement the logic for processing the voice message and transcribing it to text. There are several open-source speech-to-text libraries you can use, such as Mozilla's DeepSpeech or Kaldi.

Integrate your API with WhatsApp using the WhatsApp Business API. This will allow you to receive voice messages from WhatsApp and send transcribed text messages back to the user.

Test your API thoroughly to ensure it is functioning properly and providing accurate transcriptions.

Deploy your API to your chosen server and make it available to users.

Note that creating an API like this can be a complex and time-consuming process, and may require significant programming and technical expertise. It may be more practical to use an existing service like Twilio or Plivo, or to work with a developer or development team to build your API.

To create an API using Python with Django and deploy it on Heroku, you can follow these general steps:

Set up a Django project and create a new app for your API
Define the URL patterns for your API endpoints in the urls.py file
Define the logic for your API endpoints in the corresponding view functions
Implement the speech-to-text functionality in the view functions using a speech-to-text library or service of your choice
Set up a Heroku account and install the Heroku CLI
Create a new Heroku app and link it to your project using the Heroku CLI
Add the required dependencies and configurations to your project, including the Procfile and requirements.txt files
Push your code to the Heroku app using the Heroku CLI
Test your API by sending requests to the appropriate endpoints.
Here are some resources to help you get started:

Django documentation: https://docs.djangoproject.com/en/3.2/
Heroku documentation: https://devcenter.heroku.com/categories/reference